<a href="https://colab.research.google.com/github/AnoushkaVijay/Leukemia_GAN/blob/main/Test_Image_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#load the model
import pickle

model_name = ""
file_name = open(model_name, 'rb')
final_model = pickle.load(file_name)
print("Model is loaded")

In [ ]:
import pandas as pd
#read the data file
test_data = pd.read_csv("")
test_data.head()

In [ ]:
#label count
test_data["label"].value_counts()

In [ ]:
#labels encoding
#first saved labels for confusion matrix axis
labels = test_data["label"].unique()
labels.sort()
print(labels)

#then encode the labels
test_data['label'] = pd.factorize(test_data['label'], sort = True)[0]

In [ ]:
import numpy as np

#separating the true labels for the evaluations
y_true = np.array(test_data['label'].to_list())

In [ ]:
#getting predictions data
test_data_list = test_data.iloc[:,:-1].values

y_pred = []

for row_data in test_data_list:
  predictions = final_model.predict(np.array([row_data]))
  y_pred.append(predictions[0])

print("Predictions: {}".format(y_pred))

In [ ]:
#evaluation function
'''
def model_evaluations(y_true, y_pred, labels):
  import matplotlib.pyplot as plt
  import seaborn as sns
  from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

  acc_score = accuracy_score(y_true, y_pred)
  print("Accuracy score: {}\n".format(acc_score))

  print("Classification Report: {}".format(classification_report(y_true, y_pred)))

  plt.figure(figsize = (10,10))
  sns.heatmap(confusion_matrix(y_true, y_pred),  annot = True, fmt="g", cmap = "Blues", xticklabels = labels, yticklabels = labels)
  plt.title("Consfuion Matrix")
  plt.show()

  return classification_report(y_true, y_pred, output_dict = True)
'''

Evaluation Function | Confusion Matrix in Percentages

In [ ]:
def model_evaluations(y_true, y_pred, labels):
  import matplotlib.pyplot as plt
  import seaborn as sns
  from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

  acc_score = accuracy_score(y_true, y_pred)
  print("Accuracy score: {}\n".format(acc_score))

  print("Classification Report: {}".format(classification_report(y_true, y_pred)))

  # Compute normalized confusion matrix
  cm = confusion_matrix(y_true, y_pred, normalize='true')

    # Convert to percentages
  cm_percentage = cm * 100

    # Create annotations for confusion matrix values with the percentage sign
  annotations = np.array([[f'{val:.2f}%' for val in row] for row in cm_percentage])

  plt.figure(figsize = (10,10))
  sns.heatmap(cm_percentage,  annot = annotations, fmt="", cmap = "Blues", xticklabels = labels, yticklabels = labels)
  plt.title("Consfuion Matrix")
  plt.show()

  return classification_report(y_true, y_pred, output_dict = True)

In [ ]:
#getting evaulations
report = model_evaluations(y_true, y_pred, labels)

### Save results to a dataframe

In [ ]:
#save the report as dataframe and set the indexes
report_df = pd.DataFrame(report).transpose()
labels_list = list(labels)
labels_list.extend(["accuracy","macro avg", "weighted avg"])
print(labels_list)

In [ ]:
#save the dataframe
report_df["Labels"] = labels_list
report_df.set_index("Labels")
report_df.to_csv("final_report.csv", index = False)